## Gmail 전송

### 서비스 등록 및 API 설정

- Google API를 사용하려면 Console에서 Project를 등록해야한다.

![](https://github.com/mewithai/LangChain/blob/master/images/20240617104804.png?raw=true)

- 생성된 Project에서 `Enable API`해서 `Gmail API`를 선택
- API 안에서 `Credentials`을 `json`타입으로 다운 받을 것
  - Credentials: 신임장

![](./images/20240617105404.png)
<img src="images/20240617105404.png" />

### `Credentials`를 활용한 OAuth 로그인

In [ ]:
class GmailWrapper():
    def __init__(self) -> None:
        # If modifying these SCOPES, delete the file token.json.
        SCOPES = ["https://www.googleapis.com/auth/gmail.send"]
        creds = None

        # The file token.json stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first time.
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json')
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.json', 'w') as token:
                token.write(creds.to_json())

        self.service = build('gmail', 'v1', credentials=creds)
        
    def SendEmail(self, to, subject, content):
        self.message = MIMEText(content)
        self.message['to'] = to # 'rlawlstn@gmail.com'
        self.message['subject'] = subject  # 'Email Subject'
        
        create_message = {'raw': base64.urlsafe_b64encode(self.message.as_bytes()).decode()}

        try:
            message = (self.service.users().messages().send(userId="me", body=create_message).execute())
            print(F'sent message to {message} Message Id: {message["id"]}')
        except HTTPError as error:
            print(F'An error occurred: {error}')
            message = None

    def sendMarkdownEmail(self, to, subject, contents):
        # Get the directory of the current script
        script_dir = os.path.dirname(os.path.realpath(__file__))

        # Join the script directory with the filename
        css_file_path = os.path.join(script_dir, "EMailCSS.css")

        # Read CSS from the file
        with open(css_file_path, 'r') as f:
            css = f.read()
        md_content = contents

        # Create a Markdown renderer
        markdown = mistune.create_markdown(plugins=["table"])

        # Get Yesterday date string like 2024-05-14
        yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

        # Convert Markdown to HTML
        md_content = f"# 업Needs 파악을 위한 업계동향 정보 제공 {yesterday} 기준\n" + md_content
        html_content = markdown(md_content) 
        
        # 날짜.섹터, 

        md_with_css = css + "\n" + html_content
        message = MIMEText(md_with_css, 'html')

        message['to'] = to
        message['subject'] = subject

        raw_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}
        try:
            message = (self.service.users().messages().send(userId="me", body=raw_message).execute())
            logger.info(f'Sent message to {to}. Message Id: {message["id"]}')
        except HTTPError as error:
            logger.error(f'An error occurred: {error}')


### Init()으로 tokens 획득

- Class의 Init은 초기 설정을 하는 곳으로 `Credentials`을 사용하여 인증 `Tokens`를 생성한다.
- tokens이 있으면 그냥쓰고 없으면 Credentials를 활용하여 재생성
  - `self.service = build('gmail', 'v1', credentials=creds)`이 생성
  
### 그 이후

- 그 이후는 self.service를 가지고 활용하면 됨